In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_file = '~/Desktop/Proyecto-Final-KeepCoding/raw/airbnb-listings.csv'
cleaned_file = '~/Desktop/Proyecto-Final-KeepCoding/raw/airbnb-listings_cleaned.csv'
df = pd.read_csv(raw_file, delimiter=";", low_memory=False)

# Filtrar resultados de Madrid

In [3]:
madrid_condition = df['State'].astype(str).str.contains('Madrid')
df = df[madrid_condition]

# Limpieza de columnas

In [ ]:
columns_to_keep = ['ID', 'Host ID', 'Host Since', 'Street', 'Neighbourhood', 'Neighbourhood Cleansed', 'City', 'State', 'Zipcode', 'Latitude', 'Longitude', \
                   'Amenities', 'Property Type', 'Room Type', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type', 'Square Feet', 'Price', 'Cleaning Fee', \
                   'Availability 365', 'Review Scores Location', 'Cancellation Policy', 'Accommodates', 'Reviews per Month', 'Minimum Nights', 'Price', 'Monthly Price', 'Weekly Price']
df = df[columns_to_keep]

print(df.columns)

# Normalización del código postal

In [ ]:
replace_values = {'nan': np.nan, '-': np.nan, '28': np.nan, '-' : np.nan, '2802\n28012' : '28012', '28002\n28002': '28002', '28051\n28051' : '28051', \
                  'Madrid 28004': '28004', '2815' : '28015', '2805' : '28005'}

df = df.replace({'Zipcode': replace_values})

# Conversión del ID de la entrada en numérico en vez de string

In [ ]:
df['ID'].astype(int)

# Conversión de las fechas de 'Host Since' en date

In [ ]:
host_since = df['Host Since']
host_since = list(map(pd.to_datetime, host_since))
df['Host Since'] = host_since

# Limpio bien los nombres de los barrios: me quedo con los valores de 'Neighbourhood' excepto en el caso de los nulls que los sustituyo por el valor de 'Neighbourhood Cleansed'

In [ ]:
df['Neighbourhood'].isna().value_counts()

df['Neighbourhood'] = df['Neighbourhood'].fillna(df['Neighbourhood Cleansed'])
df = df.drop('Neighbourhood Cleansed', axis = 1)

df['Neighbourhood'].isna().value_counts()

# Property Type
## Agrupar los 'Property Type' por los que interesan u otros

In [ ]:
valid_property_types = ['House', 'Apartment', 'Bed & Breakfast', 'Condominium', 'Loft', 'Chalet', 'Hostal']

property_types = df['Property Type']
property_types = property_types.map(lambda value: value if value in valid_property_types else 'Other')
df['Property Type'] = property_types

df['Property Type'].value_counts()

# Amenities

In [ ]:
amenities_df = df['Amenities'].map(lambda value: str(value or ''))
valid_amenities = ['TV', 'Internet', 'Kitchen']

for index, amenities in enumerate(amenities_df):
    amenities = amenities.split(',')
    amenities = filter(lambda value: value in valid_amenities, amenities)
    for amenitie in amenities:
        if amenitie not in valid_amenities:
            continue
        if amenitie not in df.columns:
            df[amenitie] = False
        df.loc[index, amenitie] = True
        df.loc[index, 'Amenities Count'] = len(list(amenities))

df[valid_amenities]

# Maxs and function

In [ ]:
max_bathrooms = 3
max_bedrooms = 3
max_beds = 7

def format_max(max_value):
    return lambda value: str(value) if value < max_value else f'{max_value}+'

def calculate_ocupacy(reviews_month, min_nights, availability):
    return reviews_month * min_nights * 12 / 365

# Bathrooms

In [ ]:
bathrooms = df['Bathrooms']
bathrooms = bathrooms.map(format_max(max_bathrooms))
df['Bathrooms'] = bathrooms

In [ ]:
df['Bathrooms'].value_counts().sort_index().plot()

# Bedrooms

In [ ]:
bedrooms = df['Bedrooms']
bedrooms = bedrooms.map(format_max(max_bedrooms))
df['Bedrooms'] = bedrooms

In [ ]:
df['Bedrooms'].value_counts().sort_index().plot()

# Beds

In [ ]:
beds = df['Beds']
beds = beds.map(format_max(max_beds))
df['Beds'] = beds

In [ ]:
df['Beds'].value_counts().sort_index().plot()

# https://medium.datadriveninvestor.com/airbnb-listings-analysis-in-toronto-october-2018-2a5358bae007
# Ocupacy

In [ ]:
availability = df['Availability 365'].fillna(0)
df['Ocupacy'] = calculate_ocupacy(df['Reviews per Month'], df['Minimum Nights'], df['Availability 365'])

steps = np.arange(0, 1.01, 0.1)
groups = pd.cut(df['Ocupacy'], steps)
df.groupby(groups).size().plot()

# Cleaning Fee

In [ ]:
df['Cleaning Fee'] = df['Cleaning Fee'].fillna(0)
df['Cleaning Fee'].value_counts()

steps = 10
stept = df['Cleaning Fee'].max() / steps
steps = np.arange(0, df['Cleaning Fee'].max(), stept)
groups = pd.cut(df['Cleaning Fee'], steps)
df.groupby(groups).size().plot()

In [ ]:
df['Cleaning Fee'].value_counts().sort_index().plot()

In [ ]:
average_price = df.groupby('Neighbourhood')